In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Params", pytorch_total_params)

Total Params 269322


In [3]:
learning_rate = 1e-4
batch_size = 64
epochs = 5

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss for backprop
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation by setting grad to zero, calculating using backprop engine and stepping (using learning rate)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    # No gradient on training data (faster computation and no optimization happening here anyway)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.301226  [    0/60000]
loss: 2.304625  [ 6400/60000]
loss: 2.302113  [12800/60000]
loss: 2.302691  [19200/60000]
loss: 2.293957  [25600/60000]
loss: 2.307393  [32000/60000]
loss: 2.300146  [38400/60000]
loss: 2.301223  [44800/60000]
loss: 2.296563  [51200/60000]
loss: 2.303366  [57600/60000]
Test Error: 
 Accuracy: 17.2%, Avg loss: 0.036080 

Epoch 2
-------------------------------
loss: 2.297183  [    0/60000]
loss: 2.300837  [ 6400/60000]
loss: 2.297827  [12800/60000]
loss: 2.299071  [19200/60000]
loss: 2.290740  [25600/60000]
loss: 2.303207  [32000/60000]
loss: 2.296405  [38400/60000]
loss: 2.296880  [44800/60000]
loss: 2.292530  [51200/60000]
loss: 2.299277  [57600/60000]
Test Error: 
 Accuracy: 21.4%, Avg loss: 0.036010 

Epoch 3
-------------------------------
loss: 2.293091  [    0/60000]
loss: 2.296446  [ 6400/60000]
loss: 2.294162  [12800/60000]
loss: 2.296314  [19200/60000]
loss: 2.285428  [25600/60000]
loss: 2.299858  [32000/600

KeyboardInterrupt: 